In [ ]:

import pandas as pd
import numpy as np
import os
import skimage
import matplotlib.pyplot as plt

from pathlib import Path

import matplotlib.image as mpimg
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset,DataLoader
from PIL import Image
import torchvision.models 
import torch
import torch
import pandas as pd
import numpy as np
import warnings

import os
import re
import shutil
import string

from nltk.corpus import stopwords
from collections import Counter
import nltk
import pandas as pd
import numpy as np

import sklearn
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader
warnings.simplefilter("ignore")
nltk.download('stopwords')
!python -V

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Python 3.10.11


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)
%cd gdrive/MyDrive

Mounted at /content/gdrive/
/content/gdrive/MyDrive


In [ ]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

Is the GPU available? True


# Orig CSV

In [ ]:
orig_data=pd.read_csv("sentiment.csv")
orig_data=orig_data.iloc[:,1:]
orig_data

,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...
39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


Extracting Present images names

In [ ]:
sentim=orig_data.copy()

In [ ]:
image_Matrix=[]
image_names=[]
image_labels=[]
image_count=0
folder_path ="sentiment_images"
for index in range(len(orig_data)):
    try:
        image_path = os.path.join(folder_path, orig_data.iloc[index]["filename"])
        image=skimage.io.imread(image_path,as_gray=True)
        image=skimage.transform.resize(image,(130,130))
        image_Matrix.append(image)
        image_names.append(orig_data.iloc[index]["filename"])
        image_labels.append(orig_data.iloc[index]["sentiment"])
        #print(image_count)
    except Exception as e:
        print("Error reading image:", e)
        sentim=sentim.drop(index)


In [ ]:
sentim

,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...
39194,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39195,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39196,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39197,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


Saving clean file

In [ ]:
sentim=sentim.to_csv("clean_sentim.csv",index=False)

In [ ]:
sentim=pd.read_csv("clean_sentim.csv")
sentim

,imgid,split,filename,successful,tokens,word_sentiment,sentiment,raw
0,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'delicious', 'food', 'inc...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0]",1,a plate of delicious food including French fries.
1,31369,train,COCO_val2014_000000389081.jpg,1,"['french', 'fries', 'are', 'not', 'a', 'health...","[0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0,...",1,French fries are not a healthy food but it is ...
2,31369,train,COCO_val2014_000000389081.jpg,1,"['the', 'plate', 'has', 'one', 'of', 'my', 'fa...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0,...",1,"The plate has one of my favorite foods on it, ..."
3,31369,train,COCO_val2014_000000389081.jpg,1,"['it', 'was', 'disgusting', 'food', 'not', 'ju...","[0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,"It was disgusting food, not just bad food."
4,31369,train,COCO_val2014_000000389081.jpg,1,"['a', 'plate', 'of', 'disgusting', 'food', 'fo...","[0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 0.0, 0.0]",0,A plate of disgusting food found at a diner.
...,...,...,...,...,...,...,...,...
39104,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...
39105,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 0.0, 0.0]",0,A dirty bathroom that has a window in it.
39106,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'towel', 'that', 'is', 'on', 'a', 'rack'...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,a towel that is on a rack in a dirty bathroom
39107,24628,test,COCO_val2014_000000190705.jpg,1,"['a', 'dirty', 'bathroom', 'that', 'has', 'a',...","[0.0, 1, 1, 0.0, 0.0, 0.0, 1, 1, 0.0, 0.0, 1, 1]",0,A dirty bathroom that has a dirty window made ...


Reading Images

In [ ]:
class loadeingImages(Dataset):
    def __init__(self, sentiment, folder_images, img_transformation=None):
        self.sentiment = sentiment
        self.folder_images = folder_images
        self.img_transformation = img_transformation

    def __getitem__(self, index):
        filename = self.sentiment.iloc[index]['filename']
        path = os.path.join(self.folder_images, filename)
        img = Image.open(path).convert('RGB')
        label = self.sentiment.iloc[index]['sentiment']
        if self.img_transformation is not None:
            img = self.img_transformation(img)
            img = img.to(torch.float32)
        return img, label

    def __len__(self):
        return len(self.sentiment)

img_transformation = transforms.Compose([
    transforms.Resize((256, 256)), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

bs=64

img_train = sentim[sentim['split'] == "train"]
img_test = sentim[sentim['split'] =="test"]
img_valid = sentim[sentim['split'] =="val"]

img_train_dataset = loadeingImages(img_train, "sentiment_images", img_transformation=img_transformation)
img_test_dataset = loadeingImages(img_test, "sentiment_images", img_transformation=img_transformation)
img_valid_dataset = loadeingImages(img_valid, "sentiment_images", img_transformation=img_transformation)


dataloader_train_img = DataLoader(img_train_dataset, batch_size=bs, shuffle=False,pin_memory=True)
dataloader_img_test = DataLoader(img_test_dataset, batch_size=bs, shuffle=False,pin_memory=True)
dataloader_valid_img = DataLoader(img_valid_dataset, batch_size=bs, shuffle=False,pin_memory=True)

print("----------------------------Batch sizes------------------------------\n")
print(dataloader_train_img.batch_size)
print(dataloader_img_test.batch_size)
print(dataloader_valid_img.batch_size)
print("----------------------------dataloader length------------------------------\n")
print(len(dataloader_train_img))
print(len(dataloader_img_test))
print(len(dataloader_valid_img))

----------------------------Batch sizes------------------------------

64
64
64
----------------------------dataloader length------------------------------

302
249
61


Resnet50 Feature Extraction

In [ ]:
import torch
import torchvision

count=0
flag=0

def feature_extraction(img_dataloader, feature_extractor, device):
    image_features = []
    labels_list = []

    global count
    global flag
    with torch.no_grad():
        for images, labels in img_dataloader:
            images = images.to(device)
            labels = labels.to(device)

            features = feature_extractor(images)
            image_features.append(features.cpu())  
            labels_list.append(labels)
            count+=1
            
        flag+=count
        count=0

    
    image_features = torch.cat(image_features, dim=0)
    labels_list = torch.cat(labels_list, dim=0)

    return image_features, labels_list


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)




feature_extractor = torchvision.models.resnet50(pretrained=True)
feature_extractor.eval()
feature_extractor = feature_extractor.to(device)

count,flag= 0 ,0  
train_img_features = feature_extraction(dataloader_train_img, feature_extractor, device)
print("Loaded train images",flag)

count,flag= 0 ,0 
test_img_features = feature_extraction(dataloader_img_test, feature_extractor, device)
print("Loaded test images",flag)

count,flag= 0 ,0 
valid_img_features = feature_extraction(dataloader_valid_img, feature_extractor, device)
print("Loaded validation images",flag)


Device: cuda


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loaded train images 302
Loaded test images 249
Loaded validation images 61


In [ ]:
print(len(train_img_features[0]))
valid_img_features

19307


(tensor([[-1.8770, -1.9443, -3.4827,  ..., -0.3353,  1.4439,  4.2301],
         [-1.8770, -1.9443, -3.4827,  ..., -0.3353,  1.4439,  4.2301],
         [-1.8770, -1.9443, -3.4827,  ..., -0.3353,  1.4439,  4.2301],
         ...,
         [-3.6081, -1.9855, -0.9675,  ..., -4.4434, -0.3517,  2.2787],
         [-3.6081, -1.9855, -0.9675,  ..., -4.4434, -0.3517,  2.2787],
         [-3.6081, -1.9855, -0.9675,  ..., -4.4434, -0.3517,  2.2787]]),
 tensor([1, 1, 1,  ..., 0, 0, 0], device='cuda:0'))

In [ ]:
print(sentim[sentim["split"]=="train"].count())
print(sentim[sentim["split"]=="test"].count())
print(sentim[sentim["split"]=="val"].count())

imgid             19307
split             19307
filename          19307
successful        19307
tokens            19307
word_sentiment    19307
sentiment         19307
raw               19307
dtype: int64
imgid             15912
split             15912
filename          15912
successful        15912
tokens            15912
word_sentiment    15912
sentiment         15912
raw               15912
dtype: int64
imgid             3890
split             3890
filename          3890
successful        3890
tokens            3890
word_sentiment    3890
sentiment         3890
raw               3890
dtype: int64


Saving Features

In [ ]:
torch.save(train_img_features, 'train_img_features.pt')
torch.save(test_img_features,  'test_img_features.pt')
torch.save(valid_img_features,  'valid_img_features.pt')

Load Features

Load Gpu


In [ ]:
train_img_features = torch.load('train_img_features.pt')
test_img_features = torch.load('test_img_features.pt')
valid_img_features = torch.load('valid_img_features.pt')

Load cpu

In [ ]:
train_img_features = torch.load('train_img_features.pt',map_location=torch.device('cpu'))
test_img_features = torch.load('test_img_features.pt',map_location=torch.device('cpu'))
valid_img_features = torch.load('valid_img_features.pt',map_location=torch.device('cpu'))

img feat length

In [ ]:
print(len(train_img_features[0]))
print(len(test_img_features[0]))
print(len(valid_img_features[0]))

19307
15912
3890


TEXT

In [ ]:
sentim["raw"]=sentim["raw"].str.lower()
sentim["raw"].str.islower().all()

True

In [ ]:
stop = stopwords.words("english")
sentim["raw"]=sentim["raw"].apply(lambda words: ' '.join(word.lower() for word in words.split() if word not in stop))

In [ ]:
sentim["raw"]= sentim["raw"].str.replace('[^\w\s]','')
sentim.iloc[0]["raw"]

'plate delicious food including french fries'

In [ ]:
train_text=sentim[sentim["split"]=="train"]
print(train_text)
valid_text=sentim[sentim["split"]=="val"]
print(valid_text)
test_text=sentim[sentim["split"]=="test"]
print(test_text)

Max Lengths

In [ ]:
train_text['words_token_len'] = train_text['raw'].apply(lambda x:len(str(x).split())) 

In [ ]:
test_text['words_token_len'] = test_text['raw'].apply(lambda x:len(str(x).split())) 

In [ ]:
valid_text['words_token_len'] = valid_text['raw'].apply(lambda x:len(str(x).split())) 

In [ ]:
max_test=test_text['words_token_len'].max()
print("max length of test data ",max_test)
max_valid=valid_text['words_token_len'].max()
print("max length of valid data ",max_valid)
max_train=train_text['words_token_len'].max()
print("max length of train data ",max_train)

max length of test data  50
max length of valid data  18
max length of train data  20


Avail Gpu

In [ ]:
gpu_avail = torch.cuda.is_available()
print(f"Is the GPU available? {gpu_avail}")

Is the GPU available? True


In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

Device cuda


Making Training ,Testing and Valid data

In [ ]:
X_train=train_text["raw"].tolist()
Y_train=train_text["sentiment"].tolist()
X_test=test_text["raw"]
Y_test=test_text["sentiment"]
V_x=valid_text["raw"]
V_y=valid_text["sentiment"]
data_train=list(zip(Y_train,X_train))
data_test=list(zip(Y_test,X_test))
data_valid=list(zip(V_y,V_x))

In [ ]:
data_test[0]

(1, 'person wearing beautiful dress umbrella walking grass')

Tokenizing The text

In [ ]:
tokenizer = get_tokenizer('basic_english')
train_iterator=data_train
def token(iteration_data):
    for _, text in iteration_data:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(token(train_iterator), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

Comverting text to there indexes

In [ ]:
text_flow = lambda x: vocab(tokenizer(x))
label_flow = lambda x: int(x) 

Function that Take batches and returns there labels,offsets,features tensors

In [ ]:
def collate_batch(batch):
    label_store = []
    text_store = []
    img_feature_store = []
    img_label_store = []
    offsets = [0]
    for ((label1, text1), (label2, img_features2)) in batch:
        label_store.append(label1)
        index_of_texts = torch.tensor(text_flow(text1), dtype=torch.int64)
        text_store.append(index_of_texts)
        img_feature_store.append(torch.tensor(img_features2))  
        img_label_store.append(label2)
        offsets.append(index_of_texts.size(0))

    label_store = torch.tensor(label_store, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_store = torch.cat(text_store)
    img_feature_store = torch.stack(img_feature_store)
    img_label_store = torch.tensor(img_label_store, dtype=torch.int64)

    label_store = label_store.to(device)
    text_store = text_store.to(device)
    img_feature_store = img_feature_store.to(device)
    img_label_store = img_label_store.to(device)
    #print(label_store.shape)
    return label_store, text_store, img_feature_store, img_label_store, offsets


Multimodal images+text

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, img_feature_dim, num_class):
        super(TextClassificationModel, self).__init__()
        
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.text_l1 = nn.Linear(embed_dim, 256)  
        self.text_l2 = nn.Linear(256, 128)  
        self.text_l3 = nn.Linear(128, 64)  
        self.text_l4 = nn.Linear(64, 32)  

        self.img_l1 = nn.Linear(img_feature_dim, 2000) 
        self.img_l2 = nn.Linear(2000, 1000)
        self.img_l3 = nn.Linear(1000, 500)
        self.img_l4 = nn.Linear(500, 250)

        self.fusion_l1 = nn.Linear(282, 504)  
        self.fusion_l2 = nn.Linear(504, 252)
        self.fusion_l3 = nn.Linear(252, 126)
        self.fusion_l4 = nn.Linear(126, 64)

        self.output_combine = nn.Linear(64, num_class)

        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.text_l1.weight.data.uniform_(-initrange, initrange)
        self.text_l1.bias.data.zero_()
        self.text_l2.weight.data.uniform_(-initrange, initrange)
        self.text_l2.bias.data.zero_()
        self.text_l3.weight.data.uniform_(-initrange, initrange)
        self.text_l3.bias.data.zero_()
        self.text_l4.weight.data.uniform_(-initrange, initrange)
        self.text_l4.bias.data.zero_()

        self.img_l1.weight.data.uniform_(-initrange, initrange)
        self.img_l1.bias.data.zero_()
        self.img_l2.weight.data.uniform_(-initrange, initrange)
        self.img_l2.bias.data.zero_()
        self.img_l3.weight.data.uniform_(-initrange, initrange)
        self.img_l3.bias.data.zero_()
        self.img_l4.weight.data.uniform_(-initrange, initrange)
        self.img_l4.bias.data.zero_()

        self.fusion_l1.weight.data.uniform_(-initrange, initrange)
        self.fusion_l1.bias.data.zero_()
        self.fusion_l2.weight.data.uniform_(-initrange, initrange)
        self.fusion_l2.bias.data.zero_()
        self.fusion_l3.weight.data.uniform_(-initrange, initrange)
        self.fusion_l3.bias.data.zero_()
        self.fusion_l4.weight.data.uniform_(-initrange, initrange)
        self.fusion_l4.bias.data.zero_()

        self.output_combine.weight.data.uniform_(-initrange, initrange)
        self.output_combine.bias.data.zero_()

    def forward(self, text, img_features, offsets):
        
        
        embedded = self.embedding(text, offsets)
        text = F.relu(self.text_l1(embedded))
        text = F.relu(self.text_l2(text))
        text = F.relu(self.text_l3(text))
        text = F.relu(self.text_l4(text))

        img = F.relu(self.img_l1(img_features))
        img = F.relu(self.img_l2(img))
        img = F.relu(self.img_l3(img))
        img = F.relu(self.img_l4(img))

        fusion = torch.cat((text, img), dim=1)

        fusion = F.relu(self.fusion_l1(fusion))
        fusion = F.relu(self.fusion_l2(fusion))
        fusion = F.relu(self.fusion_l3(fusion))
        fusion = F.relu(self.fusion_l4(fusion))

        output = (self.output_combine(fusion))
        
        return output




Defining classes,features,emb dims

In [ ]:

train_iter = data_train
label,text=zip(*train_iter)
sets=set(label)
num_class = len(sets)
print(num_class)
v_size = len(vocab)
embed_size = 128
img_feat=1000
model = TextClassificationModel(v_size, embed_size,img_feat, num_class).to(device)


2


In [ ]:
pip install torchviz

In [ ]:

import torch.nn.utils as utils
from torchviz import make_dot
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader, TensorDataset
import time


Training

In [ ]:
def training(dataloader, model, criterion, optimizer):
    model.train()
    current_acc = 0.0
    total_count = 0.0
    log_interval = 100
    start_time = time.time()
    epoch_loss = 0.0
    loss_count=0
    end_of_epoch_loss=0
    for idx, (label, text, img_features, img_labels, offsets) in enumerate(dataloader):
        label = label.to(device)
        text = text.to(device)
        img_features = img_features.to(device)
        offsets = offsets.to(device)

        optimizer.zero_grad()
        predicted_label = model(text, img_features, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), 0.1)
        optimizer.step()

        max_predicted_each_class = predicted_label.argmax(1)
        compare = max_predicted_each_class == label
        correct_pred = compare.sum().item()
        current_acc += correct_pred
        total_count += label.size(0)

        loss_count += loss.item() * label.size(0)
        epoch_loss += loss.item() * label.size(0)


        if idx % log_interval == 0:
            if idx > 0:
                elapsed = time.time() - start_time
                current_loss = loss_count / total_count  # Calculate the average loss per example
                print('< epoch {:3d} < {:5d}/{:5d} batches '
                      '< accuracy {:8.3f} <loss {:.2f}'.format(ep, idx, len(dataloader),
                                                  current_acc / total_count, current_loss))
                current_acc = 0
                total_count = 0
                loss_count = 0
                start_time = time.time()

    epoch_loss /= len(dataloader.dataset)
    end_of_epoch_loss = epoch_loss
    print("end of epoch loss Train {:.2f}".format(end_of_epoch_loss))
    


Evaluation validity

In [ ]:
def evaluate(dataloader, model, criterion):
    model.eval()
    predicted_labels = []
    true_labels = []
    current_accu = 0.0
    total_count = 0.0
    acc_prop = 0
    loss_count=0
    
    with torch.no_grad():
        for id, (label, text, img_features, img_labels, offsets) in enumerate(dataloader):
            label = label.to(device)
            text = text.to(device)
            img_features = img_features.to(device)
            offsets = offsets.to(device)

            predicted_label = model(text, img_features, offsets)
            loss = criterion(predicted_label, label)
            max_predicted_each_class = predicted_label.argmax(1)
            compare = max_predicted_each_class == label
            correct_pred = compare.sum().item()
            current_accu += correct_pred
            total_count += label.size(0)
            loss_count += loss.item() * label.size(0)

            predicted_labels.extend(max_predicted_each_class.tolist())
            true_labels.extend(label.tolist())

    valid_loss=loss_count / len(dataloader.dataset)
    acc_prop = current_accu / total_count
    #print('epoch validation loss {:.3f}'.format(loss_count / len(dataloader.dataset)) )
    return valid_loss,acc_prop, predicted_labels, true_labels


Data Loader

In [ ]:
learn_rate = 5
size_of_batch = 64
epoc = 10
non = None

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learn_rate)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)
current_accuracy = non

train_text = data_train
test_text = data_test
valid_text = data_valid

train_img_labels = train_img_features[1].tolist()
train_img_f =  train_img_features[0].tolist()
train_img = [(label, feature) for label, feature in zip(train_img_labels, train_img_f)]

test_img_labels = test_img_features[1].tolist()
test_img_f =  test_img_features[0].tolist()
test_img = [(label, feature) for label, feature in zip(test_img_labels, test_img_f)]

valid_img_labels = valid_img_features[1].tolist()
valid_img_f =  valid_img_features[0].tolist()
valid_img = [(label, feature) for label, feature in zip(valid_img_labels, valid_img_f)]

train_data = list(zip(train_text, train_img))
test_data= list(zip(test_text, test_img))
valid_data= list(zip(valid_text, valid_img))



valid_iterator = DataLoader(valid_data, batch_size=size_of_batch, shuffle=True, collate_fn=collate_batch)
train_iterator = DataLoader(train_data, batch_size=size_of_batch, shuffle=True, collate_fn=collate_batch)
test_iterator = DataLoader(test_data, batch_size=size_of_batch, shuffle=True, collate_fn=collate_batch)

Running Epochs

In [ ]:
from sklearn.metrics import f1_score,accuracy_score


# Training loop
#end_of_epoch_loss = 0
predicted_labels_all = []
true_labels_all = []

for ep in range(1, epoc):
    #end_of_epoch_loss = 0
    starting_time = time.time()
    training(train_iterator, model, criterion, optimizer)
    validate_loss,validate_accuracy,predicted_labels_epoch, true_labels_epoch = evaluate(valid_iterator, model, criterion)

   
    if current_accuracy is not None:
        if current_accuracy > validate_accuracy:
            scheduler.step()
    else:
        current_accuracy = validate_accuracy

    print('=' * 70)
    print('< end of epoch {:3d} < time: {:5.2f}s < '
          'valid accuracy {:8.3f} '.format(ep, time.time() - starting_time, validate_accuracy))
    print("<valid loss {:.3f}".format (validate_loss))
    print('=' * 70)
    

    predicted_labels_all.extend(predicted_labels_epoch)
    true_labels_all.extend(true_labels_epoch)


f1 = f1_score( true_labels_all ,predicted_labels_all, average='macro')
print("-------------------- F1 score--------------------",f1)
accuracy = accuracy_score(true_labels_all, predicted_labels_all)
print("-------------------- Accuracy --------------------",accuracy )





< epoch   1 <   100/  302 batches < accuracy    0.508 <loss 0.72
< epoch   1 <   200/  302 batches < accuracy    0.508 <loss 0.72
< epoch   1 <   300/  302 batches < accuracy    0.505 <loss 0.72
end of epoch loss Train 0.72
< end of epoch   1 < time:  5.14s < valid accuracy    0.538 
<valid loss 0.728
< epoch   2 <   100/  302 batches < accuracy    0.499 <loss 0.72
< epoch   2 <   200/  302 batches < accuracy    0.504 <loss 0.72
< epoch   2 <   300/  302 batches < accuracy    0.520 <loss 0.71
end of epoch loss Train 0.72
< end of epoch   2 < time:  5.77s < valid accuracy    0.538 
<valid loss 0.726
< epoch   3 <   100/  302 batches < accuracy    0.511 <loss 0.72
< epoch   3 <   200/  302 batches < accuracy    0.513 <loss 0.72
< epoch   3 <   300/  302 batches < accuracy    0.511 <loss 0.72
end of epoch loss Train 0.72
< end of epoch   3 < time:  4.49s < valid accuracy    0.462 
<valid loss 0.710
< epoch   4 <   100/  302 batches < accuracy    0.534 <loss 0.69
< epoch   4 <   200/  302 

In [ ]:
print('Checking the results of test dataset')
loss,test_acc, predicted_labels, true_labels = evaluate(test_iterator, model, criterion)
print('test accuracy {:.3f}'.format(test_acc))

Checking the results of test dataset
test accuracy 0.548


In [ ]:
loss,test_acc, predicted_labels_test, true_labels_test = evaluate(test_iterator, model, criterion)
f1 = f1_score( true_labels_test , predicted_labels_test)
print("-------------------- F1 score--------------------",f1)
accuracy = accuracy_score(true_labels, predicted_labels)
print("-------------------- Accuracy --------------------",accuracy )

-------------------- F1 score-------------------- 0.7077898329879314
-------------------- Accuracy -------------------- 0.5480769230769231


Save Model

In [ ]:

model=TextClassificationModel(3260,128,1000,2)
torch.save(model.state_dict(), 'imgtextmulti')

Load Model

In [ ]:
model=TextClassificationModel(3260,128,1000,2)
model.load_state_dict(torch.load("imgtextmulti"))

<All keys matched successfully>

Evaluating

In [ ]:
model.to(device)
model.eval()
loss,test_acc, predicted_labels_test, true_labels_test = evaluate(test_iterator, model, criterion)
f1 = f1_score( true_labels_test , predicted_labels_test)
print("-------------------- F1 score--------------------",f1)
accuracy = accuracy_score(true_labels, predicted_labels)
print("-------------------- Accuracy --------------------",accuracy )

-------------------- F1 score-------------------- 0.7067889908256881
-------------------- Accuracy -------------------- 0.5480769230769231


In [ ]:
print(model)

TextClassificationModel(
  (embedding): EmbeddingBag(3260, 128, mode='mean')
  (text_l1): Linear(in_features=128, out_features=256, bias=True)
  (text_l2): Linear(in_features=256, out_features=128, bias=True)
  (text_l3): Linear(in_features=128, out_features=64, bias=True)
  (text_l4): Linear(in_features=64, out_features=32, bias=True)
  (img_l1): Linear(in_features=1000, out_features=2000, bias=True)
  (img_l2): Linear(in_features=2000, out_features=1000, bias=True)
  (img_l3): Linear(in_features=1000, out_features=500, bias=True)
  (img_l4): Linear(in_features=500, out_features=250, bias=True)
  (fusion_l1): Linear(in_features=282, out_features=504, bias=True)
  (fusion_l2): Linear(in_features=504, out_features=252, bias=True)
  (fusion_l3): Linear(in_features=252, out_features=126, bias=True)
  (fusion_l4): Linear(in_features=126, out_features=64, bias=True)
  (output_combine): Linear(in_features=64, out_features=2, bias=True)
)


Save Dataloader test

In [ ]:
torch.save(test_iterator,"testdataloader")

Load Dataloader test

In [ ]:
test_iterator=torch.load("testdataloader")

In [1]:
print("Thankyou")

Thankyou
